In [1]:
import sys
sys.path.append("C:/Users/ADMIN/anaconda3/envs/leeyc/Lib/site-packages")

In [2]:
# 패키지 불러오기
import os
import numpy as np
import pandas as pd
import torch

import copy
import time
from torch.utils.data import Dataset, DataLoader, ConcatDataset, Subset
from torch._utils import _accumulate
#from sklearn.metrics import precision_recall_fscore_support as score

import torch.nn as nn

In [3]:
path = "./2020_e4/user01_label.npy"

label = np.load(path)
len(label)
label #0~4

array([[0],
       [0],
       [0],
       ...,
       [3],
       [0],
       [0]])

In [6]:
# SensorTrainDataset 함수 활용 

class SensorTrainDataset(Dataset):
    """ Sensor dataset for training."""
    # Initialize data (pre-processing)
    def __init__(self):
        self.len = cv_train_dataset.shape[0]
        self.x_data = torch.from_numpy(cv_train_dataset).float()
        self.y_data = torch.from_numpy(cv_train_labels)
        print(self.x_data.shape)
        print(self.y_data.shape)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

#User List
USER_TRAIN= [
    "user01",
    "user02",
    "user03"
]

path = "./2020_e4/"

#target domain 설정
for j in range(len(USER_TRAIN)):
    target = [USER_TRAIN[j]]
    print("target=",target)
    source = [x for i,x in enumerate(USER_TRAIN) if i!=j]
    print("source=",source)
    
    #target path
    t_data = path + str(target) + '_e4.npy'
    t_label = path + str(target) + '_label.npy'
    test_data = np.load(f_data)
    test_label = np.load(f_label)
    
    
    #source domain인 경우, 데이터 로드
    for user_train in source:
        #print(user_train)
        data = path + str(user_train) + '_e4.npy'
        label = path + str(user_train) + '_label.npy'
        #데이터 로드
        train_dataset = np.load(data)
        train_label = np.load(label)

        #print(len(train_dataset))
        idx_cnt = 0
        for idx in range(len(train_dataset)):
            one_x = train_dataset[idx][:-5, 0]
            one_y = train_dataset[idx][:-5, 1]
            one_z = train_dataset[idx][:-5, 2]
            new_x = np.reshape(one_x, (5, 15))  # reshape to 5 by 15 matrix
            new_y = np.reshape(one_y, (5, 15))
            new_z = np.reshape(one_z, (5, 15))
            fin_d = np.concatenate((new_x, new_y, new_z), axis=0)  # concatenate 2D arrays
            fin_d = np.reshape(fin_d, (1, 15, 15))
            one_l = train_label[idx, 0]

            if idx_cnt == 0:
                cv_train_dataset = fin_d
                cv_train_labels = one_l
            elif idx_cnt == 1:  # second iteration
                cv_train_dataset = np.array([cv_train_dataset, fin_d])  # stack 3D arrays to 4D
                cv_train_labels = np.append(cv_train_labels, one_l)
            else:
                fin_d = np.reshape(fin_d, (1, 1, 15, 15))
                cv_train_dataset = np.concatenate((cv_train_dataset, fin_d))
                cv_train_labels = np.append(cv_train_labels, one_l)
            idx_cnt += 1
        
        # Tensor
        x_data = torch.from_numpy(cv_train_dataset).float()
        y_data = torch.from_numpy(cv_train_labels)
        print(x_data.shape)
        print(y_data.shape)
        
        #dset = SensorTrainDataset()
        
 

target= ['user01']
source= ['user02', 'user03']
torch.Size([11536, 1, 15, 15])
torch.Size([11536])
torch.Size([28455, 1, 15, 15])
torch.Size([28455])
target= ['user02']
source= ['user01', 'user03']
torch.Size([18863, 1, 15, 15])
torch.Size([18863])
torch.Size([28455, 1, 15, 15])
torch.Size([28455])
target= ['user03']
source= ['user01', 'user02']
torch.Size([18863, 1, 15, 15])
torch.Size([18863])
torch.Size([11536, 1, 15, 15])
torch.Size([11536])


SensorTrainDataset 함수 사용하지 않고 

In [7]:
#User List
USER_TRAIN= [
    "user01",
    "user02",
    "user03"
]

#path = "./2020_e4/"

def hierarchical_dataset(path):

    #target domain 설정
    for j in range(len(USER_TRAIN)):
        target = [USER_TRAIN[j]]
        print("target=",target)
        source = [x for i,x in enumerate(USER_TRAIN) if i!=j]
        print("source=",source)

        #source domain인 경우, 데이터 로드
        for user_train in source:
            #print(user_train)
            data = path + str(user_train) + '_e4.npy'
            label = path + str(user_train) + '_label.npy'
            #데이터 로드
            train_dataset = np.load(data)
            train_label = np.load(label)

            #print(len(train_dataset))
            idx_cnt = 0
            for idx in range(len(train_dataset)):
                one_x = train_dataset[idx][:-5, 0]
                one_y = train_dataset[idx][:-5, 1]
                one_z = train_dataset[idx][:-5, 2]
                new_x = np.reshape(one_x, (5, 15))  # reshape to 5 by 15 matrix
                new_y = np.reshape(one_y, (5, 15))
                new_z = np.reshape(one_z, (5, 15))
                fin_d = np.concatenate((new_x, new_y, new_z), axis=0)  # concatenate 2D arrays
                fin_d = np.reshape(fin_d, (1, 15, 15))
                one_l = train_label[idx, 0]

                if idx_cnt == 0:
                    cv_train_dataset = fin_d
                    cv_train_labels = one_l
                elif idx_cnt == 1:  # second iteration
                    cv_train_dataset = np.array([cv_train_dataset, fin_d])  # stack 3D arrays to 4D
                    cv_train_labels = np.append(cv_train_labels, one_l)
                else:
                    fin_d = np.reshape(fin_d, (1, 1, 15, 15))
                    cv_train_dataset = np.concatenate((cv_train_dataset, fin_d))
                    cv_train_labels = np.append(cv_train_labels, one_l)
                idx_cnt += 1

            # Tensor
            x_data = torch.from_numpy(cv_train_dataset).float()
            y_data = torch.from_numpy(cv_train_labels)
            print(x_data.shape)
            print(y_data.shape)

        return x_data,y_data

In [22]:
#Target select함수 추가해서 사용

#User List
USER_TRAIN= [
    "user01",
    "user02",
    "user03"
]

global target
global source

def target_select(target):
    #target domain 설정
    target_select = [target]
    print("target= ",target_select)
    source = []
    for j in range(len(USER_TRAIN)):
        if target != USER_TRAIN[j]:
            source.append(USER_TRAIN[j])
    print("source= ",source)
        
    return target,source

#target_list=[]
#source_list=[]

def hierarchical_dataset(target):
    #경로 지정
    path="./2020_e4/"
    train_dataset_list = []

    #target,source domain 설정
    target,source = target_select(target)
    
    #target data load
    target_data = path + str(target) + '_e4.npy'
    target_label = path + str(target) + '_label.npy'
    
    #데이터 로드
    valid_dataset = np.load(target_data)
    valid_label = np.load(target_label)
    #print(len(train_dataset))
    idx_cnt = 0
    len(valid_dataset)
    for idx in range(len(valid_dataset)):
        one_x = valid_dataset[idx][:-5, 0]
        one_y = valid_dataset[idx][:-5, 1]
        one_z = valid_dataset[idx][:-5, 2]
        new_x = np.reshape(one_x, (5, 15))  # reshape to 5 by 15 matrix
        new_y = np.reshape(one_y, (5, 15))
        new_z = np.reshape(one_z, (5, 15))
        fin_d = np.concatenate((new_x, new_y, new_z), axis=0)  # concatenate 2D arrays
        fin_d = np.reshape(fin_d, (1, 15, 15))
        one_l = valid_label[idx, 0]

        if idx_cnt == 0:
            cv_valid_dataset = fin_d
            cv_valid_labels = one_l
        elif idx_cnt == 1:  # second iteration
            cv_valid_dataset = np.array([cv_valid_dataset, fin_d])  # stack 3D arrays to 4D
            cv_valid_labels = np.append(cv_valid_labels, one_l)
        else:
            fin_d = np.reshape(fin_d, (1, 1, 15, 15))
            cv_valid_dataset = np.concatenate((cv_valid_dataset, fin_d))
            cv_valid_labels = np.append(cv_valid_labels, one_l)
        idx_cnt += 1

    # Tensor
    print("tersor 변환 전 후 비교")
    print(cv_valid_dataset)
    print(cv_valid_labels)
    target_x_data = torch.from_numpy(cv_valid_dataset).float()
    target_y_data = torch.from_numpy(cv_valid_labels)
    print("target_x_data.shape: ",target_x_data.shape) 
    print("target_y_data.shape: ",target_y_data.shape)
    target_dataset = (target_x_data,target_y_data)
    print(target_dataset)
    #target_list.append(target_data)
    
    
    #source data load
    for user_train in source:
        #print(user_train)
        data = path + str(user_train) + '_e4.npy'
        label = path + str(user_train) + '_label.npy'
        #데이터 로드
        train_dataset = np.load(data)
        train_label = np.load(label)

        #print(len(train_dataset))
        idx_cnt = 0
        len(train_dataset)
        for idx in range(len(train_dataset)):
            one_x = train_dataset[idx][:-5, 0]
            one_y = train_dataset[idx][:-5, 1]
            one_z = train_dataset[idx][:-5, 2]
            new_x = np.reshape(one_x, (5, 15))  # reshape to 5 by 15 matrix
            new_y = np.reshape(one_y, (5, 15))
            new_z = np.reshape(one_z, (5, 15))
            fin_d = np.concatenate((new_x, new_y, new_z), axis=0)  # concatenate 2D arrays
            fin_d = np.reshape(fin_d, (1, 15, 15))
            one_l = train_label[idx, 0]

            if idx_cnt == 0:
                cv_train_dataset = fin_d
                cv_train_labels = one_l
            elif idx_cnt == 1:  # second iteration
                cv_train_dataset = np.array([cv_train_dataset, fin_d])  # stack 3D arrays to 4D
                cv_train_labels = np.append(cv_train_labels, one_l)
            else:
                fin_d = np.reshape(fin_d, (1, 1, 15, 15))
                cv_train_dataset = np.concatenate((cv_train_dataset, fin_d))
                cv_train_labels = np.append(cv_train_labels, one_l)
            idx_cnt += 1
            
        #source dataset 한개로 저장
        #뒤에서 concat하는것 같ㅇ.. 빼도될듯?
        train_dataset = (cv_train_dataset,cv_train_labels)
       
        train_dataset_list.append(train_dataset)
            
    concat_traindataset = ConcatDataset(train_dataset_list)
    print("===================================================================================")
    print("target_dataset:",target_dataset)
    print("source_dataset:",train_dataset)
    print(len(concat_traindataset))
    
    print("---------------------------------")
    print(target_dataset) 
    print(target_dataset[0])  #sensor 
    print(target_dataset[1])  #label
    print("---------------------------------")
    print(train_dataset_list)    
    #print(len(train_dataset_list[0]))
    print(train_dataset_list[0])
    print(train_dataset_list[0][0]) #sensor 
    print(train_dataset_list[0][1]) #label
    #print(len(train_dataset_list[1]))
    #print(train_dataset_list[1])
    

    return target_dataset,train_dataset_list[0]
    #target_dataset[0]으로 하면 sensor(15*15)로 들어가서 각 데이터를 하나씩 불러오는 것이 가능
    #그 대신 label이랑 짝을 이루지 않음
    #target_dataset[1]은 label
    
    #target_dataset로 하면 sensor,label을 모두 불러오지만 하나씩 불러오지 못함 데이터 개수가 2개(sensor,label을 통으로 불러옴)로 인식
    
    #train_dataset_list[0]해서 리스트 하나 풀고 user를 for문으로 불러와서 붙이도록 해야할 듯
    
    #그 전에 (sensor,label)을 짝을 이뤄서 불러올 수 있도록 하는 것이 중요!! ***
    
    

#target이랑 source를 다르게 해줘야함 
#target은 sensor,label다 출력되는지 확인, source는 user 별로 불러와서 합쳐줘야함

#roader될 때, sensor,label이 세트로 불러와져야함..

In [23]:
#target_select('user01')
 
valid_dataset,train_dataset = hierarchical_dataset('user01')   # target을 user01로 지정

target=  ['user01']
source=  ['user02', 'user03']
tersor 변환 전 후 비교
[[[[-0.09375   -0.109375  -0.296875  ... -0.078125  -0.0625
     0.375    ]
   [ 0.765625   0.53125   -0.390625  ... -0.78125   -0.75
    -0.984375 ]
   [-1.015625  -1.078125  -0.984375  ... -0.8125    -0.859375
    -0.9375   ]
   ...
   [-0.421875  -0.390625  -0.5859375 ... -0.34375   -0.34375
    -0.25     ]
   [-0.265625  -0.3125    -0.21875   ...  0.625      0.828125
     1.015625 ]
   [ 1.046875   0.96875    1.34375   ...  0.28125    0.15625
    -0.140625 ]]]


 [[[-0.65625   -0.796875  -0.8125    ... -1.3125    -1.25
    -0.765625 ]
   [-0.65625   -0.765625  -0.625     ... -0.375     -0.203125
    -0.28125  ]
   [-0.4375    -0.828125  -0.984375  ... -1.        -1.09375
    -0.75     ]
   ...
   [ 0.578125   0.578125   0.28125   ... -0.234375  -0.28125
    -0.46875  ]
   [-0.9375    -1.015625  -1.046875  ... -0.453125  -0.21875
    -0.109375 ]
   [ 0.203125   0.34375    0.390625  ...  0.28125    0.296875
     0.5  

In [65]:
#source domain 사용 비율 정하기 _ Batch_Balanced_Dataset 

total_number_dataset = len(train_dataset)
print("hierarchical_dataset output:")
print("_dataset: ",train_dataset)
print("dataset len(image): ",total_number_dataset)

number_dataset = int(total_number_dataset * float(0.5)) # 사용비율
print("사용 비율: ",number_dataset)

dataset_split = [number_dataset, total_number_dataset - number_dataset] 
indices = range(total_number_dataset)

_dataset, _label = [Subset(train_dataset, indices[offset - length:offset])
                           for offset, length in zip(_accumulate(dataset_split), dataset_split)]
print(_dataset)
print(_label)

hierarchical_dataset output:
_dataset:  (array([[[[-0.421875, -0.421875, -0.40625 , ..., -0.40625 , -0.40625 ,
          -0.421875],
         [-0.40625 , -0.40625 , -0.40625 , ..., -0.40625 , -0.421875,
          -0.421875],
         [-0.40625 , -0.421875, -0.40625 , ..., -0.421875, -0.40625 ,
          -0.40625 ],
         ...,
         [ 0.375   ,  0.375   ,  0.375   , ...,  0.375   ,  0.375   ,
           0.375   ],
         [ 0.375   ,  0.375   ,  0.375   , ...,  0.375   ,  0.375   ,
           0.375   ],
         [ 0.375   ,  0.375   ,  0.375   , ...,  0.375   ,  0.375   ,
           0.375   ]]],


       [[[-0.40625 , -0.40625 , -0.421875, ..., -0.40625 , -0.40625 ,
          -0.421875],
         [-0.421875, -0.40625 , -0.40625 , ..., -0.40625 , -0.40625 ,
          -0.40625 ],
         [-0.40625 , -0.40625 , -0.40625 , ..., -0.40625 , -0.40625 ,
          -0.40625 ],
         ...,
         [ 0.375   ,  0.375   ,  0.375   , ...,  0.375   ,  0.375   ,
           0.375   ],
       

In [43]:
loader = torch.utils.data.DataLoader(valid_dataset,batch_size=96,shuffle=True)  # target domain
loader2 = torch.utils.data.DataLoader(train_dataset,batch_size=96,shuffle=True)  # source domain

soure domain -> list 형태 확인

next, iter 오류에서 데이터의 개수가 1개라고함 - 데이터가 하나만 들어갔는지 확인해본 결과 그건 아니고, 리스트 안에 저장된 것.
list안에 각 source domain이 어떤식으로 저장되는지 확인

In [6]:
#source domain 결합 

source=['user02','user03']
train_dataset_list = []

for user_train in source:
        #경로 지정
        path="./2020_e4/"
        #print(user_train)
        data = path + str(user_train) + '_e4.npy'
        label = path + str(user_train) + '_label.npy'
        #데이터 로드
        train_dataset = np.load(data)
        train_label = np.load(label)

        #print(len(train_dataset))
        idx_cnt = 0
        for idx in range(len(train_dataset)):
            one_x = train_dataset[idx][:-5, 0]
            one_y = train_dataset[idx][:-5, 1]
            one_z = train_dataset[idx][:-5, 2]
            new_x = np.reshape(one_x, (5, 15))  # reshape to 5 by 15 matrix
            new_y = np.reshape(one_y, (5, 15))
            new_z = np.reshape(one_z, (5, 15))
            fin_d = np.concatenate((new_x, new_y, new_z), axis=0)  # concatenate 2D arrays
            fin_d = np.reshape(fin_d, (1, 15, 15))
            one_l = train_label[idx, 0]

            if idx_cnt == 0:
                cv_train_dataset = fin_d
                cv_train_labels = one_l
            elif idx_cnt == 1:  # second iteration
                cv_train_dataset = np.array([cv_train_dataset, fin_d])  # stack 3D arrays to 4D
                cv_train_labels = np.append(cv_train_labels, one_l)
            else:
                fin_d = np.reshape(fin_d, (1, 1, 15, 15))
                cv_train_dataset = np.concatenate((cv_train_dataset, fin_d))
                cv_train_labels = np.append(cv_train_labels, one_l)
            idx_cnt += 1
        
        # Tensor
        #print("tersor 변환 전 후 비교")
        #print(cv_valid_dataset)
        #print(cv_valid_labels)
        #source_x_data = torch.from_numpy(cv_train_dataset).float()
        #source_y_data = torch.from_numpy(cv_train_labels)
        #print("source_x_data.shape: ",source_x_data.shape) 
        #print("source_y_data.shape: ",source_y_data.shape)
        train_dataset = (cv_train_dataset,cv_train_labels)
        print(train_dataset)
        #target_list.append(target_data)

        
        #source dataset 한개로 저장
        #뒤에서 concat하는것 같ㅇ.. 빼도될듯?
        print("-----------------------------------------------------------------------------")
        print("user:",user_train)
        #train_dataset = (cv_train_dataset,cv_train_labels)
        print(len(train_dataset))
        print(train_dataset)
        print("=============================================================================")
        
        #리스트 추가
        train_dataset_list.append(train_dataset)
        print("append count:",len(train_dataset_list))
        
                    
    #concat_traindataset = ConcatDataset(train_dataset_list)
    #print(len(concat_traindataset))

(array([[[[-0.765625 , -0.71875  , -0.671875 , ..., -0.734375 ,
          -0.734375 , -0.71875  ],
         [-0.65625  , -0.65625  , -0.6875   , ..., -0.625    ,
          -0.421875 , -0.4375   ],
         [-0.359375 , -0.078125 , -0.21875  , ..., -0.9375   ,
          -1.       , -0.203125 ],
         ...,
         [ 0.4375   ,  0.65625  ,  0.609375 , ...,  0.28125  ,
           0.578125 ,  1.125    ],
         [ 0.671875 ,  0.78125  ,  0.71875  , ..., -0.515625 ,
          -0.234375 ,  0.40625  ],
         [ 1.046875 ,  1.0625   ,  1.390625 , ...,  0.75     ,
           0.859375 ,  1.03125  ]]],


       [[[-0.1015625,  0.078125 , -0.140625 , ...,  0.125    ,
           0.09375  ,  0.1875   ],
         [ 0.265625 ,  0.171875 ,  0.15625  , ...,  0.25     ,
           0.09375  ,  0.078125 ],
         [ 0.09375  ,  0.078125 ,  0.       , ..., -0.453125 ,
          -0.765625 , -0.8125   ],
         ...,
         [ 0.765625 ,  0.78125  ,  0.859375 , ...,  0.453125 ,
           0.40625  , 

In [23]:
print("source domain 합치기")
sensor = [train_dataset_list[0][0],train_dataset_list[1][0]]
print("=================================================================")
print(train_dataset_list[0][0])
print("=================================================================")
print(train_dataset_list[1][0])
print("=================================================================")
print(sensor)
#sensor = torch.from_numpy(sensor).float()
#print(sensor.shape)
#print(sensor)
label = [train_dataset_list[0][1],train_dataset_list[1][1]]
print(label)
print("=================================================================")
#텐서 만들기 전에 리스트로 합치고 텐서로 만들던지 텐서로 만들지 말던지-텐서로 안만들어도 이미 배열임 그래서 안쪽에서 해야할듯
#그리고 지금은 두개밖에 없으니까 이런식으로 만들수 있지만 소스 유저가 많은 경우는 이런식으로 할 수 없음

print(len(train_dataset_list))  # 2개(user02,user03) - list
print(len(train_dataset_list[0]))  # 2개(sensor,label)  - tuple
print(len(train_dataset_list[0][0]))  # matrix(15*15) - np.array
print(len(train_dataset_list[0][1]))  # label 
print(train_dataset_list[0][1])
print(len(train_dataset_list[1]))
print(len(train_dataset_list[1][0]))  # matrix(15*15)
print(len(train_dataset_list[1][1]))  # label 

source domain 합치기
[[[[-0.765625  -0.71875   -0.671875  ... -0.734375  -0.734375
    -0.71875  ]
   [-0.65625   -0.65625   -0.6875    ... -0.625     -0.421875
    -0.4375   ]
   [-0.359375  -0.078125  -0.21875   ... -0.9375    -1.
    -0.203125 ]
   ...
   [ 0.4375     0.65625    0.609375  ...  0.28125    0.578125
     1.125    ]
   [ 0.671875   0.78125    0.71875   ... -0.515625  -0.234375
     0.40625  ]
   [ 1.046875   1.0625     1.390625  ...  0.75       0.859375
     1.03125  ]]]


 [[[-0.1015625  0.078125  -0.140625  ...  0.125      0.09375
     0.1875   ]
   [ 0.265625   0.171875   0.15625   ...  0.25       0.09375
     0.078125 ]
   [ 0.09375    0.078125   0.        ... -0.453125  -0.765625
    -0.8125   ]
   ...
   [ 0.765625   0.78125    0.859375  ...  0.453125   0.40625
    -0.28125  ]
   [-1.515625   0.90625    0.09375   ... -1.015625  -1.234375
    -1.140625 ]
   [-0.984375  -1.0625    -1.1875    ...  0.484375   0.34375
     0.15625  ]]]


 [[[-0.03125   -0.0859375 -0.45312

In [21]:
print(type(train_dataset_list[0]))
print(type(train_dataset_list[0][0]))
print(type(train_dataset_list))

<class 'tuple'>
<class 'numpy.ndarray'>
<class 'list'>


메모리 부족 문제 발생

In [35]:
#User List
USER_TRAIN= [
    "user01",
    "user02",
    "user03"
]

global target
global source

def target_select(target):
    #target domain 설정
    target_select = [target]
    print("target= ",target_select)
    source = []
    for j in range(len(USER_TRAIN)):
        if target != USER_TRAIN[j]:
            source.append(USER_TRAIN[j])
    print("source= ",source)
        
    return target,source


path="./2020_e4/"
target_dataset_list = []
target,source = target_select('user01')


#target data load
target_data = path + str(target) + '_e4.npy'
target_label = path + str(target) + '_label.npy'

#데이터 로드
valid_dataset = np.load(target_data)
valid_label = np.load(target_label)
print(len(valid_dataset))

idx_cnt = 0
for idx in range(len(valid_dataset)):
    one_x = valid_dataset[idx][:-5, 0]
    one_y = valid_dataset[idx][:-5, 1]
    one_z = valid_dataset[idx][:-5, 2]
    new_x = np.reshape(one_x, (5, 15))  # reshape to 5 by 15 matrix
    new_y = np.reshape(one_y, (5, 15))
    new_z = np.reshape(one_z, (5, 15))
    fin_d = np.concatenate((new_x, new_y, new_z), axis=0)  # concatenate 2D arrays
    fin_d = np.reshape(fin_d, (1, 15, 15))
    one_l = valid_label[idx, 0]
    #print(fin_d.shape)
    #print(one_l.shape)
    #print("\n")
    #print(fin_d)
    #print(one_l)
    
    target_dataset = (fin_d[0],one_l)
    target_dataset_list.append(target_dataset)
    
    #print("="*80)
    #target_dataset = (cv_valid_dataset,cv_valid_labels)
    #target_dataset_list.append(target_dataset)
        
    idx_cnt += 1

print(len(target_dataset_list))

target=  ['user01']
source=  ['user02', 'user03']
18863
18863


In [46]:
print(len(target_dataset_list))    # 데이터 셋 수
print(len(target_dataset_list[0]))   # sensor,label
print(target_dataset_list[0])   
#print(target_dataset_list[1])
#print(target_dataset_list[30])

print(target_dataset_list[0][0].shape)  # sensor 
print(target_dataset_list[0][1].shape)  # label


18863
2
(array([[-0.09375  , -0.109375 , -0.296875 , -0.203125 , -0.0625   ,
        -0.015625 , -0.015625 , -0.1875   , -0.171875 , -0.140625 ,
        -0.140625 , -0.078125 , -0.078125 , -0.0625   ,  0.375    ],
       [ 0.765625 ,  0.53125  , -0.390625 , -0.796875 , -0.984375 ,
        -1.484375 , -1.640625 , -1.265625 , -0.984375 , -0.828125 ,
        -0.59375  , -0.734375 , -0.78125  , -0.75     , -0.984375 ],
       [-1.015625 , -1.078125 , -0.984375 , -0.78125  , -0.734375 ,
        -0.75     , -0.40625  , -0.53125  , -0.53125  , -0.546875 ,
        -0.65625  , -0.796875 , -0.8125   , -0.859375 , -0.9375   ],
       [-0.921875 , -0.859375 , -0.9375   , -0.71875  , -0.390625 ,
        -0.71875  , -0.84375  , -1.3125   , -1.25     , -0.765625 ,
        -0.65625  , -0.765625 , -0.625    , -0.546875 , -0.171875 ],
       [ 0.015625 , -0.421875 , -0.375    , -0.90625  , -0.34375  ,
         0.       , -0.515625 , -0.375    , -0.203125 , -0.28125  ,
        -0.4375   , -0.828125 , -0.

In [47]:
load_target = DataLoader(target_dataset_list, batch_size = 96, shuffle = True)

In [48]:
#print(len(load_target))  #197개 
#first_batch = load_target.__iter__().__next__()
#print(next(first_batch))
#print(len(first_batch))   # 96개 - batch size

my_iterator=iter(load_target)
print(next(my_iterator))
print(next(my_iterator))


TypeError: only size-1 arrays can be converted to Python scalars

In [44]:
#이거 실행!!!! - next, iter 확인

print("------------------------------------------------------target")

print(len(load_target))  #197개 
first_batch = load_target.__iter__().__next__()
#print(next(first_batch))
print(len(first_batch))   # 96개 - batch size

#next 하면 first_batch[i]로 변경됨
print(first_batch[0].shape)
print(first_batch[0])
print(first_batch[1])

------------------------------------------------------target
197
2
torch.Size([96, 15, 15])
tensor([[[-0.5312, -0.5625, -0.5781,  ..., -0.5156, -0.5156, -0.5156],
         [-0.5156, -0.5312, -0.5156,  ..., -0.5312, -0.5312, -0.5312],
         [-0.5469, -0.5312, -0.5312,  ..., -0.5469, -0.5469, -0.5469],
         ...,
         [ 0.8281,  0.8281,  0.8281,  ...,  0.8125,  0.8125,  0.8125],
         [ 0.8125,  0.8125,  0.8125,  ...,  0.8125,  0.7969,  0.7969],
         [ 0.7969,  0.7969,  0.7969,  ...,  0.7969,  0.7969,  0.7969]],

        [[-0.8594, -0.8594, -0.8594,  ..., -0.8438, -0.8438, -0.8438],
         [-0.8594, -0.8438, -0.8594,  ..., -0.8438, -0.8438, -0.8594],
         [-0.8750, -0.8750, -0.8438,  ..., -0.8438, -0.8438, -0.8438],
         ...,
         [ 0.4688,  0.4688,  0.4688,  ...,  0.4844,  0.4844,  0.4688],
         [ 0.4688,  0.4688,  0.4688,  ...,  0.4844,  0.4844,  0.4844],
         [ 0.4844,  0.4844,  0.4844,  ...,  0.4688,  0.4844,  0.4844]],

        [[-0.5469, -0.57

In [43]:
#source domain 사용 비율 정하기 _ Batch_Balanced_Dataset 

total_number_dataset = len(target_dataset_list)
print("hierarchical_dataset output:")
print("_dataset: ",target_dataset_list)
print("dataset len(image): ",total_number_dataset)

number_dataset = int(total_number_dataset * float(0.5)) # 사용비율
print("사용 비율: ",number_dataset)

dataset_split = [number_dataset, total_number_dataset - number_dataset] 
indices = range(total_number_dataset)

_dataset, _label = [Subset(target_dataset_list, indices[offset - length:offset])
                           for offset, length in zip(_accumulate(dataset_split), dataset_split)]
print(_dataset)
print(_label)

hierarchical_dataset output:
_dataset:  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [58]:
#Target select함수 추가해서 사용

#User List
USER_TRAIN= [
    "user01",
    "user02",
    "user03"
]

global target
global source

def target_select(target):
    #target domain 설정
    target_select = [target]
    print("target= ",target_select)
    source = []
    for j in range(len(USER_TRAIN)):
        if target != USER_TRAIN[j]:
            source.append(USER_TRAIN[j])
    print("source= ",source)
        
    return target,source


def hierarchical_dataset(target):
    #경로 지정
    path="./2020_e4/"
    
    target_dataset_list = []
    source_dataset_list = []

    #target,source domain 설정
    target,source = target_select(target)
    
    #target data load
    target_data = path + str(target) + '_e4.npy'
    target_label = path + str(target) + '_label.npy'
    
    #데이터 로드
    valid_dataset = np.load(target_data)
    valid_label = np.load(target_label)
    #print(len(train_dataset))
    idx_cnt = 0
    len(valid_dataset)
    for idx in range(len(valid_dataset)):
        one_x = valid_dataset[idx][:-5, 0]
        one_y = valid_dataset[idx][:-5, 1]
        one_z = valid_dataset[idx][:-5, 2]
        new_x = np.reshape(one_x, (5, 15))  # reshape to 5 by 15 matrix
        new_y = np.reshape(one_y, (5, 15))
        new_z = np.reshape(one_z, (5, 15))
        fin_d = np.concatenate((new_x, new_y, new_z), axis=0)  # concatenate 2D arrays
        fin_d = np.reshape(fin_d, (1, 15, 15))
        one_l = valid_label[idx, 0]
    #print(fin_d.shape)
    #print(one_l.shape)
    #print("\n")
    #print(fin_d)
    #print(one_l)
    
        target_dataset = (fin_d[0],one_l)
        target_dataset_list.append(target_dataset)
    print(len(target_dataset_list))

    
    #source data load
    for user_train in source:
        #print(user_train)
        data = path + str(user_train) + '_e4.npy'
        label = path + str(user_train) + '_label.npy'
        #데이터 로드
        train_dataset = np.load(data)
        train_label = np.load(label)

        #print(len(train_dataset))
        idx_cnt = 0
        len(train_dataset)
        for idx in range(len(train_dataset)):
            one_x = train_dataset[idx][:-5, 0]
            one_y = train_dataset[idx][:-5, 1]
            one_z = train_dataset[idx][:-5, 2]
            new_x = np.reshape(one_x, (5, 15))  # reshape to 5 by 15 matrix
            new_y = np.reshape(one_y, (5, 15))
            new_z = np.reshape(one_z, (5, 15))
            fin_d = np.concatenate((new_x, new_y, new_z), axis=0)  # concatenate 2D arrays
            fin_d = np.reshape(fin_d, (1, 15, 15))
            one_l = train_label[idx, 0]

            source_dataset = (fin_d[0],one_l)       
            source_dataset_list.append(source_dataset)
    
    print(len(source_dataset_list))
    #concat_traindataset = ConcatDataset(source_dataset_list)
    

    return target_dataset_list,source_dataset_list

In [59]:
target,source = hierarchical_dataset('user01')

target=  ['user01']
source=  ['user02', 'user03']
18863
39991


In [60]:
load_target = DataLoader(target,batch_size = 96, shuffle = True)
load_source = DataLoader(source,batch_size = 96, shuffle = True)

In [67]:
#이거 실행!!!! - next, iter 확인

print("------------------------------------------------------target")

print(len(load_target))  #197개 
first_batch = load_target.__iter__().__next__()
#print(next(first_batch))
print(len(first_batch))   # 96개 - batch size

#next 하면 first_batch[i]로 변경됨
print(first_batch[0].shape)
print(first_batch[0])
print(first_batch[1])

print("------------------------------------------------------sourec")

print(len(load_source))  #197개 
first_batch2 = load_source.__iter__().__next__()
#print(next(first_batch))
print(len(first_batch2))   # 96개 - batch size

#next 하면 first_batch[i]로 변경됨
print(first_batch2[0].shape)
print(first_batch2[0])
print(first_batch2[1])


------------------------------------------------------target
197
2
torch.Size([96, 15, 15])
tensor([[[-0.8438, -0.8438, -0.8438,  ..., -0.8750, -0.9219, -0.9062],
         [-0.8906, -0.9219, -0.9062,  ..., -0.8906, -0.9688, -0.9375],
         [-0.8281, -0.8594, -0.9219,  ..., -0.9062, -0.9062, -0.9219],
         ...,
         [-0.2344, -0.2500, -0.2656,  ..., -0.1719, -0.1719, -0.1406],
         [-0.1406, -0.2031, -0.1875,  ..., -0.1562, -0.3281, -0.2812],
         [-0.1406, -0.1562, -0.1250,  ..., -0.0156, -0.0625, -0.0625]],

        [[-0.7500, -0.7500, -0.7500,  ..., -0.7500, -0.7500, -0.7500],
         [-0.7500, -0.7500, -0.7500,  ..., -0.7500, -0.7500, -0.7500],
         [-0.7500, -0.7500, -0.7500,  ..., -0.7500, -0.7500, -0.7500],
         ...,
         [-0.6562, -0.6562, -0.6562,  ..., -0.6562, -0.6562, -0.6562],
         [-0.6562, -0.6406, -0.6562,  ..., -0.6562, -0.6562, -0.6562],
         [-0.6562, -0.6562, -0.6406,  ..., -0.6562, -0.6562, -0.6406]],

        [[-0.6406, -0.79